In [242]:
# !pip3 install beautifulsoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# URL for the SeattleSeq SNV annotation tool
nucleotide_array = ['A', 'T', 'C', 'G']
url = "https://cadd.gs.washington.edu/snv"
df = pd.read_csv('O00555-variants.csv')
results_df = df.copy()
results_df['RawScore_A'] = None
results_df['PHRED_A'] = None
results_df['RawScore_T'] = None
results_df['PHRED_T'] = None
results_df['RawScore_C'] = None
results_df['PHRED_C'] = None
results_df['RawScore_G'] = None
results_df['PHRED_G'] = None

df.head()

,Chromosome,NTPosition,NTChange,AAPosition,AAChange,Phenotype
0,19,13506221,C>A,2,A2S,"Developmental and epileptic encephalopathy, 42..."
1,19,13506220,G>A,2,A2V,"Developmental and epileptic encephalopathy, 42..."
2,19,13506218,G>A,3,R3C,"Developmental and epileptic encephalopathy, 42..."
3,19,13506218,G>C,3,R3G,"Developmental and epileptic encephalopathy, 42..."
4,19,13506211,C>A,5,G5V,NaN


In [289]:
for index in range(df.shape[0]):
# Example payload for a query (this may need to be adjusted based on actual site input fields)
    payload = {
        'chr': df.loc[index].Chromosome,
        'pos': df.loc[index].NTPosition,
        'ref': df.loc[index].NTChange[0],
        'alt': df.loc[index].NTChange[2],
        'version': 'GRCh38-v1.7'
    }

    # Send the query to the website
    response = requests.post(url, data=payload)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find and extract the relevant information
        results = str(soup.find(id='snv-data')).split('\n')
        # ref_value = results[3].split('</td><td>')[1]

        alt_values = results[4].split('</td><td>')
        if len(alt_values) != 3:
            alt_values = [re.findall(r'>[A,T,C,G]<',results[4])[0][1], " ", " "]

        raw_scores = results[5].split('</td><td>')
        if len(raw_scores) != 3:
            raw_scores = [re.findall(r'\d+.\d*',results[5])[0], '-1', '-1']

        phred_scores = results[6].split('</td><td>')
        if len(phred_scores) != 3:
            phred_scores = [re.findall(r'\d+.\d*',results[6])[0],' -1', '-1']



        hashmap = {alt_values[0][-1] : [re.findall(r'\d+.*\d*',raw_scores[0])[0], re.findall(r'\d+.*\d*',phred_scores[0])[0]], 
                    alt_values[1] : [raw_scores[1], phred_scores[1]], 
                    alt_values[2][0] : [raw_scores[2].rstrip('</td></tr>'), phred_scores[2].rstrip('</td></tr>')]}
        
        hash_temp = {'A' : [-1, -1], 'T' : [-1, -1], 'C' : [-1, -1], 'G': [-1, -1]}
        hash_temp.update(hashmap)            

        results_df.loc[index, "RawScore_A"] = hash_temp['A'][0]
        results_df.loc[index, "PHRED_A"] = hash_temp['A'][1]

        results_df.loc[index, "RawScore_T"] = hash_temp['T'][0]
        results_df.loc[index, "PHRED_T"] = hash_temp['T'][1]

        results_df.loc[index, "RawScore_C"] = hash_temp['C'][0]
        results_df.loc[index, "PHRED_C"] = hash_temp['C'][1]

        results_df.loc[index, "RawScore_G"] = hash_temp['G'][0]
        results_df.loc[index, "PHRED_G"] = hash_temp['G'][1]

        print('Index ', index,' Complete')
    else:
        print(f"Error: {response.status_code}")
    
    time.sleep(5)

Index  0  Complete
Index  1  Complete
Index  2  Complete
Index  3  Complete
Index  4  Complete
Index  5  Complete
Index  6  Complete
Index  7  Complete
Index  8  Complete
Index  9  Complete
Index  10  Complete
Index  11  Complete
Index  12  Complete
Index  13  Complete
Index  14  Complete
Index  15  Complete
Index  16  Complete
Index  17  Complete
Index  18  Complete
Index  19  Complete
Index  20  Complete
Index  21  Complete
Index  22  Complete
Index  23  Complete
Index  24  Complete
Index  25  Complete
Index  26  Complete
Index  27  Complete
Index  28  Complete
Index  29  Complete
Index  30  Complete
Index  31  Complete
Index  32  Complete
Index  33  Complete
Index  34  Complete
Index  35  Complete
Index  36  Complete
Index  37  Complete
Index  38  Complete
Index  39  Complete
Index  40  Complete
Index  41  Complete
Index  42  Complete
Index  43  Complete
Index  44  Complete
Index  45  Complete
Index  46  Complete
Index  47  Complete
Index  48  Complete
Index  49  Complete
Index  50 

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [288]:
results_df.head()

,Chromosome,NTPosition,NTChange,AAPosition,AAChange,Phenotype,RawScore_A,PHRED_A,RawScore_T,PHRED_T,RawScore_C,PHRED_C,RawScore_G,PHRED_G
0,19,13506221,C>A,2,A2S,"Developmental and epileptic encephalopathy, 42...",3.606917,23.0,-1,-1,-1,-1,-1,-1
1,19,13506220,G>A,2,A2V,"Developmental and epileptic encephalopathy, 42...",4.929395,27.5,-1,-1,-1,-1,-1,-1
2,19,13506218,G>A,3,R3C,"Developmental and epileptic encephalopathy, 42...",4.646263,26.1,-1,-1,-1,-1,-1,-1
3,19,13506218,G>C,3,R3G,"Developmental and epileptic encephalopathy, 42...",None,None,None,None,None,None,None,None
4,19,13506211,C>A,5,G5V,NaN,None,None,None,None,None,None,None,None


In [148]:
results_df.to_csv("Variant Results.csv")